# Simple CNN for mnist dataset

## CNN output formula
![](./cnn_output_formula.png)


In [64]:

# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!


In [65]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [66]:


class CNN(nn.Module):
    def __init__(self,in_channels=1,num_classes=10):
        super(CNN,self).__init__()
        self.conv1=nn.Conv2d(in_channels=in_channels,out_channels=8,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.conv2=nn.Conv2d(in_channels=8,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.fc1=nn.Linear(16*7*7,num_classes)

    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=(self.pool(x))
        x=F.relu(self.conv2(x))
        x=(self.pool(x))
        x=x.reshape(x.shape[0],-1)
        x=self.fc1(x)
        return x


In [67]:
# Hyperparamters

input_size=784
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=10


In [68]:
# Loading dataset
train_dataset=datasets.MNIST(root='datasets/',train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset=datasets.MNIST(root='datasets/',train=False,transform=transforms.ToTensor(),download=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)



In [69]:
model=CNN(in_channels=1,num_classes=10).to(device=device)

In [70]:
# x=torch.randn(62,1,28,28).to(device=device)
# print(model(x).shape)

In [71]:
#Loss and Optimizer

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)


In [72]:
# Train Network



for epoch in range(num_epochs):
    for batch_idx, (data,targets) in enumerate(tqdm(train_loader)):

        #getting the data
        data=data.to(device=device)
        targets=targets.to(device=device)
        # data is the batch of data, targets is the target label of each sample in the batch


        #forward
        scores=model(data)
        loss=criterion(scores,targets)
        
        torch.set_grad_enabled(True)  # Context-manager 

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent
        optimizer.step()
    #print('epoch_num=',epoch,'   Train accuracy=',check_accuracy(model=model,loader=train_dataset))
    #print('=======================================================')

        

#check_accuracy(model,test_dataset)







100%|██████████| 938/938 [00:09<00:00, 99.51it/s] 


In [73]:
def check_accuracy(loader,model):
    num_correct=0
    num_samples=0
    model.eval()  # to turn off batch normalisation and dropout layers

    with torch.no_grad():  # to turn off backprobagation
        for x, y in loader:
            x=x.to(device=device)
            #x=x.reshape((x.shape[0],-1))

            y=torch.tensor(y).to(device=device)

            scores=model(x)
            _, predictions=scores.max(1)  #max return a tuple (max element, max element index)
            num_correct+= (predictions==y).sum()
            num_samples+= predictions.size(0)
        #print('accuracy=',((float(num_correct)/num_samples)*100) )

    model.train()   # to return model back to training mode
    return (float(num_correct)/num_samples)*100

In [74]:

print('test accuracy=',check_accuracy(loader=test_loader,model=model))

/tmp/ipykernel_88135/3799589990.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y=torch.tensor(y).to(device=device)


test accuracy= 98.5
